<a href="https://colab.research.google.com/github/venkat-narahari/phyfit-notion/blob/main/NotionPhysicalActivity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [87]:
import requests
import pandas as pd
from google.colab import userdata
from datetime import time, timedelta
import ipywidgets as widgets
from IPython.display import clear_output, display, HTML, Javascript
import matplotlib.pyplot as plt
import plotly.express as px
from ipywidgets import interact, widgets

In [3]:
# keys stored in ENV variables
api_key = userdata.get('NOTION_API_KEY')
physical_activity_log_db_id = userdata.get('PHYSICALACTIVITYLOG_DB_ID')
excercise_list_db_id = userdata.get('EXERCISELIST_DB_ID')
bodyweight_log_db_id = userdata.get('BODYWEIGHTLOG_DB_ID')

# define headers to Notion API
headers = {
    "Authorization": f"Bearer {api_key}",
    "Notion-Version": "2022-06-28",
    "Content-Type": "application/json",
}

In [4]:
# retrive data from physical activity log database
# load first page
url = f"https://api.notion.com/v1/databases/{physical_activity_log_db_id}/query"
physical_activity_log_response = requests.post(url, headers=headers)

# If you want to check if the request was successful (status code 200 OK)
if physical_activity_log_response.status_code == 200:
    physical_activity_log_response = physical_activity_log_response.json()
    # Process the data as needed
else:
    print(f"Request failed with status code {status_code}")

# check if response contains "results" key
if "results" not in physical_activity_log_response:
    physical_activity_log_response = []
    raise Exception("No results found in response")
else:
    physical_activity_log_records = physical_activity_log_response["results"]
    while physical_activity_log_response.get("has_more"):
        physical_activity_log_response = requests.post(
            f"https://api.notion.com/v1/databases/{physical_activity_log_db_id}/query",
            json={"start_cursor": physical_activity_log_response["next_cursor"]},
            headers=headers,
        ).json()
        if "results" in physical_activity_log_response:
            physical_activity_log_records.extend(physical_activity_log_response["results"])

In [5]:
# retrive data from excercise list database
# load first page
url = f"https://api.notion.com/v1/databases/{excercise_list_db_id}/query"
excercise_list_response = requests.post(url, headers=headers)

# If you want to check if the request was successful (status code 200 OK)
if excercise_list_response.status_code == 200:
    excercise_list_response = excercise_list_response.json()
    # Process the data as needed
else:
    print(f"Request failed with status code {status_code}")

# check if response contains "results" key
if "results" not in excercise_list_response:
    excercise_list_records = []
    raise Exception("No results found in response")
else:
    excercise_list_records = excercise_list_response["results"]
    while excercise_list_response.get("has_more"):
        excercise_list_response = requests.post(
            f"https://api.notion.com/v1/databases/{excercise_list_db_id}/query",
            json={"start_cursor": excercise_list_response["next_cursor"]},
            headers=headers,
        ).json()
        if "results" in excercise_list_response:
            excercise_list_records.extend(excercise_list_response["results"])

In [6]:
# retrive data from bodyweight log database
# load first page
url = f"https://api.notion.com/v1/databases/{bodyweight_log_db_id}/query"
bodyweight_log_response = requests.post(url, headers=headers)

# If you want to check if the request was successful (status code 200 OK)
if bodyweight_log_response.status_code == 200:
    bodyweight_log_response = bodyweight_log_response.json()
    # Process the data as needed
else:
    print(f"Request failed with status code {status_code}")

# check if response contains "results" key
if "results" not in bodyweight_log_response:
    bodyweight_log_records = []
    raise Exception("No results found in response")
else:
    bodyweight_log_records = bodyweight_log_response["results"]
    while bodyweight_log_response.get("has_more"):
        bodyweight_log_response = requests.post(
            f"https://api.notion.com/v1/databases/{bodyweight_log_db_id}/query",
            json={"start_cursor": bodyweight_log_response["next_cursor"]},
            headers=headers,
        ).json()
        if "results" in bodyweight_log_response:
            bodyweight_log_records.extend(bodyweight_log_response["results"])

In [7]:
# define a helper function to transform the JSON to a Pandas DF for Physical Activity Log records

def get_raw_value_pal(item, item_name):
    item_type = item['type']
    if item_name == 'Session' and len(item[item_type]) > 0 :
        return item[item_type][0]['plain_text']
    elif item_name == 'Excercise' and len(item[item_type]) > 0 :
        return item[item_type][0]['id']
    elif item_name == 'SetsxReps' and len(item[item_type]) > 0 :
        return [int(x) for x in item[item_type][0]['plain_text'].split(',')]
    elif item_name == 'Weights'and len(item[item_type]) > 0:
        return item[item_type][0]['plain_text'].split(',')
    elif item_name == 'Date' and item[item_type] is not None:
        return item[item_type]['start']
    elif item_name == 'Notes' and len(item[item_type]) > 0 :
        return item[item_type][0]['plain_text']
    else:
        return ''

In [8]:
# create Pandas DF for Physical Activity Log
pal_values = []

for record in physical_activity_log_records:
    properties = record['properties']
    pal_values.append({
        'Session': get_raw_value_pal(properties['Session'], 'Session'),
        'ExerciseID_PAL': get_raw_value_pal(properties['Exercise'], 'Excercise'),
        'SetsxReps': get_raw_value_pal(properties['SetsxReps'], 'SetsxReps'),
        'Weights': get_raw_value_pal(properties['Weights'], 'Weights'),
        'Date': get_raw_value_pal(properties['Date'], 'Date'),
        'Notes_PAL': get_raw_value_pal(properties['Notes'], 'Notes'),
    })

pal_df = pd.DataFrame(pal_values)
pal_df.columns = ['Session', 'ExerciseID_PAL', 'SetsxReps', 'Weights', 'Date', 'Notes_PAL']

print(pal_df.sort_values(by='Date', ascending=True).head(2))


      Session                        ExerciseID_PAL     SetsxReps  \
296  Session1  9f00a790-df44-4ca8-a3d7-bbbb565d0fdf  [14, 14, 14]   
284  Session1  48a97238-d37e-4d6a-9472-00391fb8bdde  [12, 12, 12]   

                Weights                           Date Notes_PAL  
296                      2024-01-01T18:00:00.000-06:00            
284  [25lb, 25lb, 25lb]  2024-01-01T18:00:00.000-06:00            


In [9]:
# define a helper function to transform the JSON to a Pandas DF for Excercise list records

def get_raw_value_el(item, item_name):
    item_type = item['type']
    if item_name == 'Name' and len(item[item_type]) > 0 :
        return item[item_type][0]['plain_text']
    elif item_name == 'MuscleGroup' and len(item[item_type]) > 0 :
        return [entry["name"] for entry in item[item_type]]
    elif item_name == 'Type' and item[item_type] is not None :
        return item[item_type]['name']
    elif item_name == 'Tools'and len(item[item_type]) > 0:
        return [entry["name"] for entry in item[item_type]]
    elif item_name == 'Notes'and len(item[item_type]) > 0:
        return item[item_type][0]['plain_text']
    else:
        return ''

In [10]:
# create Pandas DF for Exercise List
el_values = []

for record in excercise_list_records:
    properties = record['properties']
    el_values.append({
        'ExerciseID_EL': record['id'],
        'Exercise': get_raw_value_el(properties['Name'], 'Name'),
        'MuscleGroup': get_raw_value_el(properties['MuscleGroup'], 'MuscleGroup'),
        'Type': get_raw_value_el(properties['Type'], 'Type'),
        'Tools': get_raw_value_el(properties['Tools'], 'Tools'),
        'Notes_EL': get_raw_value_el(properties['Notes'], 'Notes')
    })

el_df = pd.DataFrame(el_values)
el_df.columns = ['ExerciseID_EL','Exercise', 'MuscleGroup', 'Type', 'Tools', 'Notes_EL']

print(el_df.sort_values(by='Exercise', ascending=True).head(2))

                            ExerciseID_EL Exercise MuscleGroup Type Tools  \
149  3712e584-24ab-4d8c-8296-bfde58d34c2e                                   
75   eac289ce-f9a9-4f19-99e1-7b48df2d0bc8                                   

    Notes_EL  
149           
75            


In [11]:
# define a helper function to transform the JSON to a Pandas DF for Bodyweight Log records

def get_raw_value_bw(item, item_name):
    item_type = item['type']
    if item_name == 'Type' and len(item[item_type]) > 0 :
        return item[item_type][0]['plain_text']
    elif item_name == 'Date' and item[item_type] is not None:
        return item[item_type]['start']
    elif item_name == 'Weight' and item[item_type] is not None:
        return item[item_type]
    else:
        return ''

In [12]:
# create Pandas DF for Bodyweight Log
bw_values = []

for record in bodyweight_log_records:
    properties = record['properties']
    bw_values.append({
        'Type': get_raw_value_bw(properties['Type'], 'Type'),
        'Date': get_raw_value_bw(properties['Date'], 'Date'),
        'Weight': get_raw_value_bw(properties['Weight'], 'Weight')
    })

bw_df = pd.DataFrame(bw_values)
bw_df.columns = ['Type', 'Date', 'Weight']

print(bw_df.sort_values(by='Date', ascending=True).head(2))

          Type        Date  Weight
11  BodyWeight  2024-01-01   176.5
12  BodyWeight  2024-01-02   176.5


In [92]:
# Join Physical Activity Log and Excercise list for performing analysis
pal_el_df = pd.merge(pal_df, el_df, how ='left', left_on='ExerciseID_PAL', right_on='ExerciseID_EL' )
# Format the date in the merged_df


# Identify rows with format 'YYYY-MM-DD'
rows_to_modify = pal_el_df['Date'].apply(lambda x: len(x) == 10)

indices_to_modify = rows_to_modify[rows_to_modify].index

if len(indices_to_modify) > 0:
    # Create a warning message with the indices of rows with missing time information
    warning_message = f"You have missing time information in the following rows: {', '.join(map(str, indices_to_modify))}"

    # Display the warning as a popup
    warning_popup = f"""
    <script>
        alert('{warning_message}');
    </script>
    """

    display(HTML(warning_popup))


# Modify the dates in the identified rows
pal_el_df.loc[rows_to_modify, 'Date'] = pal_el_df.loc[rows_to_modify, 'Date'] + 'T18:00:00.000-06:00'

# Convert the 'Date' column to datetime
pal_el_df['Date'] = pd.to_datetime(pal_el_df['Date'], errors='coerce')


print(pal_el_df.head(10))

     Session                        ExerciseID_PAL         SetsxReps  \
0     Rest12  01f384c6-c26a-4445-8b6b-52a48ea6c88f              [60]   
1     Rest12  4e22fbab-58e8-4bb7-9014-21abe44cc702              [15]   
2     Rest12  9f00a790-df44-4ca8-a3d7-bbbb565d0fdf  [25, 25, 25, 25]   
3     Rest12  8e4d4435-e1fe-45f9-9697-02963b3e1ff9           [14000]   
4     Rest11  9f00a790-df44-4ca8-a3d7-bbbb565d0fdf          [25, 25]   
5     Rest11  8e4d4435-e1fe-45f9-9697-02963b3e1ff9           [12000]   
6     Rest10  9f00a790-df44-4ca8-a3d7-bbbb565d0fdf  [25, 25, 25, 25]   
7     Rest10  8e4d4435-e1fe-45f9-9697-02963b3e1ff9           [12000]   
8  Session20  9f00a790-df44-4ca8-a3d7-bbbb565d0fdf      [21, 21, 21]   
9  Session20  44949927-3203-4c15-8486-70f82e9f3edf      [15, 15, 15]   

              Weights                      Date Notes_PAL  \
0                     2024-02-01 18:00:00-06:00             
1                     2024-02-01 18:00:00-06:00             
2                     20

In [93]:
data = {'Date': ['2024-02-01T17:00:00.000-06:00', '2024-02-01T18:00:00.000-06:00', '2024-02-01', '2024-01-31T18:00:00.000-06:00']}
data1 = {'Date': ['2024-02-01', '2024-02-02', '2024-02-03']}
df = pd.DataFrame(data)


# Identify rows with format 'YYYY-MM-DD'
rows_to_modify = df['Date'].apply(lambda x: len(x) == 10)

# Modify the dates in the identified rows
df.loc[rows_to_modify, 'Date'] = df.loc[rows_to_modify, 'Date'] + 'T18:00:00.000-06:00'

# Convert the 'Date' column to datetime
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')


print(df)

                       Date
0 2024-02-01 17:00:00-06:00
1 2024-02-01 18:00:00-06:00
2 2024-02-01 18:00:00-06:00
3 2024-01-31 18:00:00-06:00


In [94]:
def calculate_total_reps(df):
    # Create a total reps column per exercise per day
    df['TotalReps'] = df['SetsxReps'].apply(sum)
    return df

def get_form_values(form):
    start_date = form.children[0].value
    end_date = form.children[1].value
    return start_date, end_date

def create_date_widgets():
    # Get the current date
    current_date = pd.to_datetime('today').date()

    # Define the start and end date widgets
    start_date_widget = widgets.DatePicker(
        description='Start Date',
        value=current_date - pd.DateOffset(days=30)
    )
    end_date_widget = widgets.DatePicker(
        description='End Date',
        value=current_date
    )

    return start_date_widget, end_date_widget
'''
def create_dropdown(exercise_names):
    # Define the exercise dropdown widget
    exercise_dropdown = widgets.Dropdown(
        options=exercise_names,
        value='Walking',
        description='Select Exercise:',
        disabled=False,
    )

    return exercise_dropdown
'''

def create_multiselect_dropdown(exercise_names):
    # Define the exercise dropdown widget
    exercise_dropdown = widgets.SelectMultiple(
        options=exercise_names,
        value=[exercise_names[0]],  # Set default value to the first exercise
        description='Exercise ',
        disabled=False
    )

    return exercise_dropdown


def create_form(start_date_widget, end_date_widget, exercise_dropdown):
    # Create the form with the exercise dropdown
    form = widgets.VBox(
        children=[start_date_widget, end_date_widget, exercise_dropdown]
    )

    return form


In [95]:
def plot_exercisehistory_linechart(df, start_date, end_date, selected_exercises):
    # Get the start and end dates
    start_date = pd.to_datetime(start_date).date()
    end_date = pd.to_datetime(end_date).date()

    # Filter the data based on the start and end dates and selected exercise
    filtered_df = df[df['Date'].dt.date >= start_date]
    filtered_df = filtered_df[filtered_df['Date'].dt.date <= end_date]
    filtered_df = filtered_df[filtered_df['Exercise'].isin(selected_exercises)]
    filtered_df['Date'] = filtered_df['Date'].dt.date

    # Clear the existing output
    clear_output(wait=True)

    fig = px.line(filtered_df, x='Date', y='TotalReps', color='Exercise', markers=True, line_shape='linear', title='Exercise History', labels={'TotalReps': 'Total Reps'}, hover_data=['Exercise'])
    # Show the plot
    fig.show()

    #
    # Plot the data
    # plt.figure(figsize=(15, 3))
    # graph = plt.plot(filtered_df['Date'], filtered_df['TotalReps'], 'o--')
    # plt.xlabel('Date')
    # plt.ylabel('Total Reps')
    # plt.title(f'{selected_exercise} Exercise History')
    # plt.grid(True)
    # plt.show()

In [99]:

def print_totalreps(df, start_date, end_date, selected_exercises, form):
    # Get the start and end dates
    start_date = pd.to_datetime(start_date).date()
    end_date = pd.to_datetime(end_date).date()

    # Calculate the number of days between start and end date
    num_days = (end_date - start_date).days
    print(df['Date'].dtype);

    # Filter the data based on the start and end dates and selected exercise
    filtered_df = df[df['Date'].dt.date >= start_date]
    filtered_df = filtered_df[filtered_df['Date'].dt.date <= end_date]
    filtered_df = filtered_df[filtered_df['Exercise'].isin(selected_exercises)]
    filtered_df['Date'] = filtered_df['Date'].dt.date
    filtered_df['MaxRep'] = filtered_df['SetsxReps'].apply(lambda x: max(x) if x else None)


    # Count Rest and Session days
    rest_days_count = df[df['Session'].str.contains('Rest')]['Session'].nunique()
    sessions_count = df[df['Session'].str.contains('Session')]['Session'].nunique()

    print(f'Total Session Days: {sessions_count}')
    print(f'Total Rest Days: {rest_days_count}')
    print(f'Total Days: {num_days}')

    for exercise in selected_exercises:
      totalreps_exercise = filtered_df.loc[filtered_df['Exercise'] == exercise, 'TotalReps'].sum()
      daysdone_count = filtered_df.loc[filtered_df['Exercise'] == exercise, 'TotalReps'].count()
      maxreps_in_daterange = filtered_df.loc[filtered_df['Exercise'] == exercise, 'TotalReps'].max()
      maxrep_in_daterange = filtered_df.loc[filtered_df['Exercise'] == exercise, 'MaxRep'].max()

      print(f'{exercise}')
      print(f'Total Reps: {totalreps_exercise}')
      print(f'Avg Reps/day in date range: {totalreps_exercise/num_days}')
      print(f'Avg Reps/day in days done: {totalreps_exercise/daysdone_count}')
      print(f'Max reps in date range: {maxreps_in_daterange}')
      print(f'Max rep/set in date range: {maxrep_in_daterange}')

    display(form)


In [97]:
def observe_change(change, df, start_date_widget, end_date_widget, exercise_dropdown, form):
    # Clear the existing output
    start_date = start_date_widget.value
    end_date = end_date_widget.value
    selected_exercises = exercise_dropdown.value
    plot_exercisehistory_linechart(df, start_date, end_date, selected_exercises)
    print_totalreps(df, start_date, end_date, selected_exercises, form)

In [100]:
def plot_data(df):
    # Calculate total reps
    df = calculate_total_reps(df)

    # Extract unique exercise names from the dataframe
    exercise_names = df['Exercise'].unique()

    # Setup form, widgets and dropdowns
    start_date_widget, end_date_widget = create_date_widgets()
    exercise_dropdown = create_multiselect_dropdown(exercise_names)
    form = create_form(start_date_widget, end_date_widget, exercise_dropdown)

    # Observe changes on widgets
    start_date_widget.observe(lambda change: observe_change(change, df, start_date_widget, end_date_widget, exercise_dropdown, form), names='value')
    end_date_widget.observe(lambda change: observe_change(change, df, start_date_widget, end_date_widget, exercise_dropdown, form), names='value')
    exercise_dropdown.observe(lambda change: observe_change(change, df, start_date_widget, end_date_widget, exercise_dropdown, form), names='value')

    # Get the initial start and end dates and selected exercise
    start_date = start_date_widget.value
    end_date = end_date_widget.value
    selected_exercises = exercise_dropdown.value

    # Plot the data
    plot_exercisehistory_linechart(df, start_date, end_date, selected_exercises)

    print_totalreps(df, start_date, end_date, selected_exercises, form)

# Example usage:
# Assuming pal_el_df is your dataframe
plot_data(pal_el_df)

datetime64[ns, pytz.FixedOffset(-360)]
Total Session Days: 20
Total Rest Days: 12
Total Days: 15
Walking
Total Reps: 154000
Avg Reps/day in date range: 10266.666666666666
Avg Reps/day in days done: 9625.0
Max reps in date range: 12000
Max rep/set in date range: 12000
